In [ ]:
import cv2
import numpy as np
cap = cv2.VideoCapture("vtest.avi")
ret, frame1 = cap.read()
prvs = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(frame1)
hsv[...,1] = 255
while(1):
    ret, frame2 = cap.read()
    next = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv2.imwrite('opticalfb.png',frame2)
        cv2.imwrite('opticalhsv.png',bgr)
    prvs = next
cap.release()
cv2.destroyAllWindows()

In [4]:
import numpy as np
import cv2
cap = cv2.VideoCapture(0)
tmp = []
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 20, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7)
tmp = p0
# Create a mask image for drawing purposes
counter = 0
HEIGHT = 0
WIDTH = 0
mask = np.zeros_like(old_frame)
while(1):
    ret,frame = cap.read()
    if(counter == 0): 
        HEIGHT = frame.shape[0]
        WIDTH = frame.shape[1]
    counter += 1
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
cv2.destroyAllWindows()
cap.release()

In [9]:
print(HEIGHT,WIDTH)
# print(tmp)
print(tmp.shape)
print(type(tmp[0,0,0]))

480 640
(12, 1, 2)
<class 'numpy.float32'>


<h1>Make Grid of References</h1>

In [1]:
def calc_grid(WIDTH,HEIGHT,FINE_DEGREE):
    H_W_RATIO = HEIGHT / WIDTH
    X_DISTANCE = WIDTH // (FINE_DEGREE + 1)
    Y_DISTANCE = int(HEIGHT // ((FINE_DEGREE * H_W_RATIO) + 1))
    p0 = np.ndarray(((FINE_DEGREE * int(FINE_DEGREE * H_W_RATIO)), 1, 2))
    counter = 0
    Xs = np.arange(X_DISTANCE,X_DISTANCE * (FINE_DEGREE - 1) + 1, X_DISTANCE)
    Ys = np.arange(Y_DISTANCE,Y_DISTANCE * (int(FINE_DEGREE * H_W_RATIO) + 1), Y_DISTANCE)
    for y in Ys:
        for x in Xs:
            p0[counter,0,:] = [y,x]
            counter += 1
    return p0.astype(float)    

In [2]:
import numpy as np
import cv2
cap = cv2.VideoCapture(0)

FINE_DEGREE = 20
WIDTH = 640
HEIGHT = 480

# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 20, 0.03))
# Create some random colors
color = np.random.randint(0,255,(100,3))
# Take first frame and find corners in it
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = calc_grid(WIDTH,HEIGHT,FINE_DEGREE)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)
while(1):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    # Now update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
cv2.destroyAllWindows()
cap.release()

error: C:\projects\opencv-python\opencv\modules\video\src\lkpyramid.cpp:1244: error: (-215) (npoints = prevPtsMat.checkVector(2, CV_32F, true)) >= 0 in function cv::`anonymous-namespace'::SparsePyrLKOpticalFlowImpl::calc
